In [2]:
choices = ['Rock', 'Paper', 'Scissors', 'Lizard', 'Spock']
payoffs = {
    'Rock': 3, 'Paper': 2, 'Scissors': 1, 'Lizard': 4, 'Spock': 5
}

# Outcomes from the perspective of the first choice
outcomes = {
    ('Rock', 'Scissors'): 'win', ('Rock', 'Lizard'): 'win',
    ('Paper', 'Rock'): 'win', ('Paper', 'Spock'): 'win',
    ('Scissors', 'Paper'): 'win', ('Scissors', 'Lizard'): 'win',
    ('Lizard', 'Spock'): 'win', ('Lizard', 'Paper'): 'win',
    ('Spock', 'Scissors'): 'win', ('Spock', 'Rock'): 'win'
}

counter_moves = {
    'Rock': ['Paper', 'Spock'], 
    'Paper': ['Scissors', 'Lizard'], 
    'Scissors': ['Rock', 'Spock'], 
    'Lizard': ['Rock', 'Scissors'], 
    'Spock': ['Paper', 'Lizard']
}

lose_moves = {
    'Rock': ['Scissors', 'Lizard'], 
    'Paper': ['Rock', 'Spock'], 
    'Scissors': ['Paper', 'Lizard'], 
    'Lizard': ['Paper', 'Spock'], 
    'Spock': ['Scissors', 'Rock']
}


In [ ]:
# Example for a Random Strategy
import random
import pandas as pd

class Strategy:
    def next_move(self, history):
        raise NotImplementedError

class RandomStrategy(Strategy):
    def next_move(self, history):
        return random.choice(choices)

class FixedStrategy(Strategy):
    def __init__(self, idx=4):
        self.choices = ['Rock', 'Paper', 'Scissors', 'Lizard', 'Spock']
        self.move = self.choices[idx]
    
    def next_move(self, history):
        # Randomly choose between Lizard and Spock
        return self.move

class CopyStrategy(Strategy):
    def __init__(self, player_1_2=1):
        self.player = player_1_2
    def next_move(self, history):
        # If it's the first move, choose randomly
        if not history:
            return random.choice(choices)
        # Otherwise, copy the opponent's last move
        else:
            # history[-1][1] refers to the opponent's last move
            if self.player == 1: return history[-1][1]
            return history[-1][0]


In [4]:
class AdaptiveStrategy(Strategy):
    def __init__(self, player_1_2=1, memory_length=10):
        self.player = player_1_2
        self.memory_length = memory_length
    
    def next_move(self, history):
        # If it's the first move, or no history is available, choose randomly
        if not history:
            return random.choice(choices)
        
        # Count the frequency of the opponent's moves
        if self.player==1:
            opponent_moves = [it[1] for it in history]
        else:
            opponent_moves = [it[0] for it in history]
        if len(opponent_moves)>self.memory_length:
            opponent_moves = opponent_moves[-self.memory_length:]
        move_frequency = {move: opponent_moves.count(move) for move in choices}
        
        # Find the opponent's most frequent move
        most_frequent_move = max(move_frequency, key=move_frequency.get)
        
        # Based on the opponent's most frequent move, decide our move
        # The strategy here is to beat the most frequent move of the opponent

        return random.choice( counter_moves[most_frequent_move] )

In [5]:
class TryToLoseStrategy(Strategy):
    def __init__(self, player_1_2=1, memory_length=10):
        self.player = player_1_2
        self.memory_length = memory_length

    def next_move(self, history):
        # If it's the first move, or no history is available, choose randomly
        if not history:
            return random.choice(choices)
        
        # Determine the opponent's moves based on perspective
        if self.player == 1:
            opponent_moves = [it[1] for it in history]
        else:
            opponent_moves = [it[0] for it in history]
        
        # Limit the memory to the most recent moves if necessary
        if len(opponent_moves) > self.memory_length:
            opponent_moves = opponent_moves[-self.memory_length:]
        
        # Count the frequency of the opponent's moves
        move_frequency = {move: opponent_moves.count(move) for move in choices}
        
        # Find the opponent's most frequent move
        most_frequent_move = max(move_frequency, key=move_frequency.get)
        
        return random.choice( lose_moves[most_frequent_move] )

In [6]:
def simulate_round(strategy1, strategy2, history):
    move1 = strategy1.next_move(history)
    move2 = strategy2.next_move(history)
    # Determine outcome and update scores
    if (move1, move2) in outcomes: 
        return move1, move2, +payoffs[move1], -payoffs[move2]
    elif move1 == move2:
        return move1, move2, 0, 0
    else:
        return move1, move2, -payoffs[move1], +payoffs[move2] 

def run_simulation(strategy1, strategy2, num_rounds=100):
    history = []
    scores = [0, 0]  # [strategy1_score, strategy2_score]
    for _ in range(num_rounds):
        m1, m2, s1, s2 = simulate_round(strategy1, strategy2, history)
        scores[0] += s1
        scores[1] += s2
        history.append( (m1, m2, s1, s2) )
        # Update history and scores
    return history, scores 


## Simulations

### random vs any others

- random vs all fixed




In [8]:
total = [0, 0, 0]
player1_scores, player2_scores = [], []
player1_win, player2_win = [], []

for times in range(100):
       num_rounds = 1000
       lst = [
              ("FixedStrategy", FixedStrategy(4)), 
              ("CopyStrategy", CopyStrategy(2)), 
              ("AdaptiveStrategy", AdaptiveStrategy(2, num_rounds)),
              ("TryToLoseStrategy", TryToLoseStrategy(2, num_rounds))
              ]
       strategy1 = RandomStrategy()
       (name, strategy2) = lst[0]
       #print( "RandomStrategy vs", name )
       history, scores = run_simulation(strategy1, strategy2, num_rounds)
       #print("scores: player 1 vs player 2, ", scores)
       if scores[0]>scores[1]: total[0] += 1
       elif scores[0]<scores[1]: total[1] += 1
       else: total[2] += 1
       player1_scores.append( scores[0] )
       player2_scores.append( scores[1] )
       if scores[0] > scores[1]: 
              player1_win.append( 1 )
              player2_win.append( 0 )
       elif scores[0] < scores[1]:
              player1_win.append( 0 )
              player2_win.append( 1 )
       else: 
              player1_win.append( 0 )
              player2_win.append( 0 )

df = pd.DataFrame()   
df[ 'score1' ] = player1_scores
df[ 'score2' ] = player2_scores
df[ 'win1' ] = player1_win
df[ 'win2' ] = player2_win
#total, df['score1'].mean(), df['score2'].mean(), df['score1'].std(), df['score2'].std() 

In [10]:
# score difference through out 100 times, random vs fixed(Spock)
df['dif'] = df[ 'score1' ] - df[ 'score2' ]
df['dif_mean'] = df['dif'].mean()
#df[['dif', 'dif_mean']].plot()

### fixed vs the rest

- fixed vs copy: degrade to 1 round

In [11]:
total = [0, 0, 0]
player1_scores, player2_scores = [], []
for times in range(1000):
       num_rounds = 100
       lst = [
              ("CopyStrategy", CopyStrategy(2)), 
              ("AdaptiveStrategy", AdaptiveStrategy(2, num_rounds)),
              ("TryToLoseStrategy", TryToLoseStrategy(2, num_rounds))
              ]
       strategy1 = FixedStrategy()
       (name, strategy2) = lst[2]
       #print( "RandomStrategy vs", name )
       history, scores = run_simulation(strategy1, strategy2, num_rounds)
       #print("scores: player 1 vs player 2, ", scores)
       if scores[0]>scores[1]: total[0] += 1
       elif scores[0]<scores[1]: total[1] += 1
       else: total[2] += 1
       player1_scores.append( scores[0] )
       player2_scores.append( scores[1] )
df = pd.DataFrame()
df[ 'score1' ] = player1_scores
df[ 'score2' ] = player2_scores
#total, df['score1'].mean(), df['score2'].mean(), df['score1'].std(), df['score2'].std() 

### Copy vs the rest

In [12]:
total = [0, 0, 0]
player1_scores, player2_scores = [], []
for times in range(1000):
       num_rounds = 100
       lst = [ 
              ("AdaptiveStrategy", AdaptiveStrategy(2, num_rounds)),
              ("TryToLoseStrategy", TryToLoseStrategy(2, num_rounds))
              ]
       strategy1 = CopyStrategy(1)
       (name, strategy2) = lst[1]
       #print( "RandomStrategy vs", name )
       history, scores = run_simulation(strategy1, strategy2, num_rounds)
       #print("scores: player 1 vs player 2, ", scores)
       if scores[0]>scores[1]: total[0] += 1
       elif scores[0]<scores[1]: total[1] += 1
       else: total[2] += 1
       player1_scores.append( scores[0] )
       player2_scores.append( scores[1] )
df = pd.DataFrame()
df[ 'score1' ] = player1_scores
df[ 'score2' ] = player2_scores
#total, df['score1'].mean(), df['score2'].mean(), df['score1'].std(), df['score2'].std() 

### Adaptive vs Try to Lose

In [13]:
total = [0, 0, 0]
player1_scores, player2_scores = [], []
for times in range(1000):
       num_rounds = 100
       lst = [ 
              ("TryToLoseStrategy", TryToLoseStrategy(2, num_rounds))
              ]
       strategy1 = AdaptiveStrategy(1, num_rounds)
       (name, strategy2) = lst[0]
       #print( "RandomStrategy vs", name )
       history, scores = run_simulation(strategy1, strategy2, num_rounds)
       #print("scores: player 1 vs player 2, ", scores)
       if scores[0]>scores[1]: total[0] += 1
       elif scores[0]<scores[1]: total[1] += 1
       else: total[2] += 1
       player1_scores.append( scores[0] )
       player2_scores.append( scores[1] )
df = pd.DataFrame()
df[ 'score1' ] = player1_scores
df[ 'score2' ] = player2_scores
#total, df['score1'].mean(), df['score2'].mean(), df['score1'].std(), df['score2'].std() 

### Long vs Short Memory

In [14]:
total = [0, 0, 0]
player1_scores, player2_scores = [], []
for times in range(1000):
       num_rounds = 100
       lst = [ 
              ("AdaptiveStrategy", AdaptiveStrategy(2, 50))
              ]
       strategy1 = AdaptiveStrategy(1, num_rounds)
       (name, strategy2) = lst[0]
       #print( "RandomStrategy vs", name )
       history, scores = run_simulation(strategy1, strategy2, num_rounds)
       #print("scores: player 1 vs player 2, ", scores)
       if scores[0]>scores[1]: total[0] += 1
       elif scores[0]<scores[1]: total[1] += 1
       else: total[2] += 1
       player1_scores.append( scores[0] )
       player2_scores.append( scores[1] )
df = pd.DataFrame()
df[ 'score1' ] = player1_scores
df[ 'score2' ] = player2_scores
#total, df['score1'].mean(), df['score2'].mean(), df['score1'].std(), df['score2'].std()

In [15]:
import collections
dic1 = collections.Counter(list(map(lambda x:x[0], history)))
dic2 = collections.Counter(list(map(lambda x:x[1], history)))
#dic1, dic2 

In [17]:
import numpy as np
import matplotlib.pyplot as plt

data = []
moves_to_num = {  'Rock':1, 'Paper':2, 'Scissors':3, 'Lizard':4, 'Spock':5 }
for k in dic1:
    v1 = moves_to_num[k]
    v2 = dic1[k]
    data.append( (v1,v2) )
#plt.imshow( data )
#plt.title( "2-D Heat Map" )
#plt.show()

### Animation 

In [80]:
# https://matplotlib.org/stable/users/explain/animations/animations.html
# https://www.geeksforgeeks.org/how-to-create-animations-in-python/
from matplotlib import pyplot as plt
import matplotlib.animation as animation 
from matplotlib.animation import FuncAnimation
from matplotlib.artist import Artist
import numpy as np
import random
from IPython.display import HTML

# the index for each moves: 1-5
moves_dic = {1:'Rock', 2:'Paper', 3:'Scissors', 4:'Lizard', 5:'Spock'}
# for each move, its winning situations and corresponding scores when winning/losing
gestures = {
    "Rock": {"wins": ["Scissors", "Lizard"], "winning_points": 1, "losing_points": -1},
    "Paper": {"wins": ["Rock", "Spock"], "winning_points": 2, "losing_points": -2},
    "Scissors": {"wins": ["Paper", "Lizard"], "winning_points": 3, "losing_points": -3},
    "Lizard": {"wins": ["Spock", "Paper"], "winning_points": 4, "losing_points": -4},
    "Spock": {"wins": ["Scissors", "Rock"], "winning_points": 5, "losing_points": -5}
}

# basic information about the animation
	# tick_time: interval, namely, the delay between frames in milliseconds
	# frame_list: the text parts that will be updated for each tick
	# scores: the cumulative scores for two players
tick_time = 50
frame_list = []
scores = [0, 0]

def generate_animation(player1_moves, player2_moves, title_str, tick_time, filename, y_limit=0):
	#global scores

	# basic setting about the plot image
	figure, ax = plt.subplots() # figsize=(16,8)
	plt.title(title_str, fontsize = 20)
	plt.axhline(y=0, color='r', linestyle='-') # horizontal lines: y=0
	# labels for the x and y axis
	plt.xlabel("rounds number")
	plt.ylabel("score difference: player 1 - player 2 ")
	# Setting limits for x and y axis
	round_num = len(player1_moves)
	x_limit = round_num
	if y_limit==0: y_limit = round_num//2
	ax.set_xlim(0, x_limit)
	ax.set_ylim(-y_limit, y_limit)
	# Since plotting a single graph
	line,  = ax.plot(0, 0) 
	
	# ticks (how many rounds), score_dif (cumulative scores by each tick)
	ticks, score_dif = [], []
	
	# initialize the scores for player 1 and 2 as 0
	scores[0] = 0
	scores[1] = 0

	# animation by tick
	def animation_function(i):
		global frame_list, score1, score2
		
		# actions for 2 players at each tick
		action01 = player1_moves[i]
		action02 = player2_moves[i]
		# player 1 action vs player 2 action, winning/losing scores
		s1, s2 = 0, 0
		if action02 in gestures[action01]['wins']:
			s1 = gestures[action01]['winning_points']
			s2 = gestures[action02]['losing_points']
		elif action01 in gestures[action02]['wins']:
			s1 = gestures[action01]['losing_points']
			s2 = gestures[action02]['winning_points']
		scores[0] += s1
		scores[1] += s2 
		s1 = "+" + str(s1) if s1>0 else str(s1)
		s2 = "+" + str(s2) if s2>0 else str(s2)
		
		# clear the text for last tick
		for f in frame_list: Artist.remove(f)
		# plot the information of actions 
		f1 = ax.text(round(0.1*x_limit), round(0.5*y_limit), action01, fontsize = 20, color='maroon')
		f2 = ax.text(round(0.8*x_limit), round(0.5*y_limit), action02, fontsize = 20, color='maroon')
		f3 = ax.text(round(0.5*x_limit), round(0.5*y_limit), "vs", fontsize = 20, color='darkorange')
		# plot the information of winning/losing scores 
		f4 = ax.text(round(0.2*x_limit), round(0.65*y_limit), s1, fontsize = 15, color='red')
		f5 = ax.text(round(0.9*x_limit), round(0.65*y_limit), s2, fontsize = 15, color='red')
		# plot information of cumulative scores
		f6 = ax.text(round(0.1*x_limit), round(0.8*y_limit), "P 1: "+str(scores[0]), fontsize = 15, color='purple')
		f7 = ax.text(round(0.8*x_limit), round(0.8*y_limit), "P 2: "+str(scores[1]), fontsize = 15, color='purple')
		# plot the information of round number
		f8 = ax.text(round(0.4*x_limit), round(0.8*y_limit), "Round: " + str(i+1), fontsize = 15, color='blue')
		
		#print( "Round: " + str(i+1), action01, action02, s1, s2, "P 1: "+str(scores[0]), "P 2: "+str(scores[1]) )

		# update tick and score difference information
		ticks.append(i)
		score_dif.append(scores[0] - scores[1])
		line.set_xdata(ticks)
		line.set_ydata(score_dif)
		
		# the last round, to show which player win
		if i+1==round_num: 
			win_msg = "player 1 win!" if scores[0]>scores[1] else "player 2 win!"
			f9 = ax.text(round(0.25*x_limit), round(0.3*y_limit), win_msg, fontsize = 30, color='red')
			frame_list = [f1, f2, f3, f4, f5, f6, f7, f8, f9]
		else:
			frame_list = [f1, f2, f3, f4, f5, f6, f7, f8]
		return line,
	
	# generate animation
	ani = FuncAnimation(figure,
							  func = animation_function,
							  frames = np.arange(0, round_num, 1), 
							  interval = tick_time,
							  repeat = False)
	plt.grid() 
	#plt.show()
	# save as video
	writervideo = animation.FFMpegWriter(fps=60)
	ani.save('/Users/xiaoyao/Desktop/'+filename, writer=writervideo)
	plt.close() 
	return 

In [81]:
# random vs random
round_num = 1000
player1_moves = []
player2_moves = []
for i in range(round_num): player1_moves.append( moves_dic[random.randint(1, 5)] )
for i in range(round_num): player2_moves.append( moves_dic[random.randint(1, 5)] )
title_str = "Random vs Random"
filename = title_str + ".mp4"
generate_animation(player1_moves, player2_moves, title_str, tick_time, filename, y_limit=400) 

In [82]:
# random vs fixed-lizard
round_num = 1000
player1_moves = []
player2_moves = []
for i in range(round_num): player1_moves.append( moves_dic[random.randint(1, 5)] )
for i in range(round_num): player2_moves.append( moves_dic[4] )
title_str = "Random vs Fixed-Lizard"
filename = title_str + ".mp4"
generate_animation(player1_moves, player2_moves, title_str, tick_time, filename, y_limit=800) 

In [83]:
# random vs fixed-spock
round_num = 1000
player1_moves = []
player2_moves = []
for i in range(round_num): player1_moves.append( moves_dic[random.randint(1, 5)] )
for i in range(round_num): player2_moves.append( moves_dic[5] )
title_str = "Random vs Fixed-Spock"
filename = title_str + ".mp4"
generate_animation(player1_moves, player2_moves, title_str, tick_time, filename, y_limit=600)

In [84]:
# adaptive long vs short memory
strategy1 = AdaptiveStrategy(1, 1000) # long memory, record last 1000 steps
strategy2 = AdaptiveStrategy(2, 10) # short memory, record last 10 steps
history, scores = run_simulation(strategy1, strategy2, 1000) # run 1000 rounds

round_num = 1000
player1_moves = list(map(lambda x:x[0], history))
player2_moves = list(map(lambda x:x[1], history))
title_str = "Adaptive Long vs Short Memory"
filename = title_str + ".mp4"
generate_animation(player1_moves, player2_moves, title_str, tick_time, filename, y_limit=1200) 

In [87]:
# random vs try-to-lose
strategy1 = RandomStrategy() 
strategy2 = TryToLoseStrategy(2, 1000) # long memory, record last 1000 steps
history, scores = run_simulation(strategy1, strategy2, 1000) # run 1000 rounds

round_num = 1000
player1_moves = list(map(lambda x:x[0], history))
player2_moves = list(map(lambda x:x[1], history))
title_str = "Random vs Try-to-lose"
filename = title_str + ".mp4"
generate_animation(player1_moves, player2_moves, title_str, tick_time, filename, y_limit=300) 

In [89]:
# random vs copy
strategy1 = RandomStrategy() 
strategy2 = CopyStrategy(2) 
history, scores = run_simulation(strategy1, strategy2, 1000) # run 1000 rounds

round_num = 1000
player1_moves = list(map(lambda x:x[0], history))
player2_moves = list(map(lambda x:x[1], history))
title_str = "Random vs Copy"
filename = title_str + ".mp4"
generate_animation(player1_moves, player2_moves, title_str, tick_time, filename, y_limit=300) 